### TL;DR
[Generated music](https://soundcloud.com/kaan-berke-u-urlar/music-generated-by-ai) can be reached and listened by the link.

In [1]:
!git clone https://github.com/Skuldur/Classical-Piano-Composer
!mv /content/Classical-Piano-Composer/midi_songs /content/midi_songs
!rm -rf /content/Classical-Piano-Composer/

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 24.70 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [7]:
import glob
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from music21 import converter, instrument, note, chord, stream
import os

In [12]:
MODEL_BASE_PATH = "./models"
SEQ_LEN = 100

In [4]:
notes = []
for file in tqdm(glob.glob("midi_songs/*.mid")):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts:  # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:  # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

100%|██████████| 92/92 [00:17<00:00,  5.17it/s]


In [8]:
pitches = sorted(set(item for item in notes))
n_vocab = len(pitches)
note_to_int = dict((note, number) for number, note in enumerate(pitches))
network_input = []
network_output = []
for i in range(0, len(notes) - SEQ_LEN, 1):
    sequence_in = notes[i:i + SEQ_LEN]
    sequence_out = notes[i + SEQ_LEN]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
network_input = np.reshape(network_input, (n_patterns, SEQ_LEN, 1)).astype("float")
network_input = network_input / n_vocab
network_output = tf.keras.utils.to_categorical(network_output)

In [10]:
layers = [
    tf.keras.layers.LSTM(256,
         input_shape=(network_input.shape[1], network_input.shape[2]),
         return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(1024, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.LSTM(512),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(n_vocab),
    tf.keras.layers.Activation("softmax")
]
model = tf.keras.models.Sequential(layers=layers)

model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])


In [15]:
if not os.path.exists(MODEL_BASE_PATH):
    os.mkdir(MODEL_BASE_PATH)
model_filepath = os.path.join(MODEL_BASE_PATH, "model_{epoch:02d}_{loss:.4f}.h5")
cp = tf.keras.callbacks.ModelCheckpoint(
    model_filepath,
    monitor="loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min"
)
model.fit(network_input,
          network_output,
          epochs=100,
          batch_size=32,
          callbacks=[cp])


Epoch 1/100
1784/1784 [==============================] - 369s 207ms/step - loss: 4.5677 - acc: 0.0372

Epoch 00001: loss improved from inf to 4.56773, saving model to ./models/model_01_4.5677.h5
Epoch 2/100
1784/1784 [==============================] - 367s 206ms/step - loss: 4.5257 - acc: 0.0393

Epoch 00002: loss improved from 4.56773 to 4.52566, saving model to ./models/model_02_4.5257.h5
Epoch 3/100
  41/1784 [..............................] - ETA: 5:59 - loss: 4.5103 - acc: 0.0518

KeyboardInterrupt: ignored

In [17]:
saved_models = os.listdir(MODEL_BASE_PATH)
assert saved_models, "There is no model saved."

print(f"Models in the '{MODEL_BASE_PATH}'")
print(*zip(range(len(saved_models)), saved_models), sep="\n")
model_no = int(input("Please select the model that you want to use: "))
model.load_weights(os.path.join(MODEL_BASE_PATH, saved_models[model_no]))

Models in the './models'
(0, 'model_01_4.5677.h5')
(1, 'model_02_4.5257.h5')
Please select the model that you want to use:1


In [18]:
start = np.random.randint(0, len(network_input) - 1)
int_to_note = dict((number, note) for number, note in enumerate(pitches))
pattern = network_input[start]
prediction_output = []
for note_index in tqdm(range(750)):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

100%|██████████| 750/750 [01:37<00:00,  7.70it/s]


In [19]:
offset = 0
output_notes = []
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write("midi", fp="output.mid")

'output.mid'

In [21]:
!pip install midi2audio
!pip install fluidsynth
!pip install pyFluidSynth
!sudo apt-get install fluidsynth
!midi2audio /content/test_output.mid /content/test_output.wav

  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4514 sha256=e7b8f1ad47b85c01faa1f14064ecf035e47b6d7c73b259365d4fbc48e610a9a1
  Stored in directory: /root/.cache/pip/wheels/44/4b/81/dee2d535b2dd27af13c230ba49aa6f05a5dff0f57c68acef55
Successfully built fluidsynth
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 37 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 

In [23]:
!midi2audio /content/output.mid /content/output.wav

FluidSynth version 1.1.9
Copyright (C) 2000-2018 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Parameter '/root/.fluidsynth/default_sound_font.sf2' not a SoundFont or MIDI file or error occurred identifying it.
fluidsynth: error: Unable to open file "share/soundfonts/default.sf2"
fluidsynth: error: Couldn't load soundfont file
fluidsynth: error: Failed to load SoundFont "share/soundfonts/default.sf2"
Rendering audio to file '/content/output.wav'..
